# Stochastic Partial Derivative Equations

<!-- SUMMARY: Estimation and Simulations performed in the framework of SPDE -->

<!-- CATEGORY: SPDE -->

In this tutorial, we show how to use the API SPDE

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import numpy as np
import matplotlib.pyplot as plt

from sksparse.cholmod import cholesky
import scipy as sc
from scipy.sparse import *
from scipy.sparse.linalg import *
import numpy as np

gdoc.setNoScroll()

## Parameters

In [ ]:
# Data
np.random.seed(123)
ndat = 1000

# Model
rangev = 0.2
sill = 1.
nugget = 0.1

# Grid 
nx = [50,50]
dx = [0.02,0.02]
x0 = [0,0]

#Grid meshing
nxm = [75,75]
dxm = [0.02,0.02]
x0m = [-0.25,-0.25]

dbfmt = gl.DbStringFormat.createFromFlags(flag_stats=True, names=["spde*"])

### Grid and Meshing

In [ ]:
grid = gl.DbGrid.create(nx,dx,x0)
gridExt = gl.DbGrid.create(nxm,dxm,x0m)
mesh = gl.MeshETurbo(gridExt)

### Model

In [ ]:
model = gl.Model.createFromParam(gl.ECov.MATERN,param=1,range=rangev,sill=sill)
model.addCovFromParam(gl.ECov.NUGGET,sill=nugget)

### Data

In [ ]:
dat = gl.Db.create()
dat["x"] = np.random.uniform(size=ndat)
dat["y"] = np.random.uniform(size=ndat)
dat.setLocators(["x","y"],gl.ELoc.X)

## SPDE non-conditional simulation

### Grid query

In [ ]:
spde = gl.SPDE(model,grid,None,gl.ESPDECalcMode.SIMUNONCOND)
gl.law_set_random_seed(131351)
iuid = spde.compute(grid)
grid.display(dbfmt)

In [ ]:
gp.plot(grid)
gp.decoration(title="Non Conditional Simulation")

### Data query

In [ ]:
gl.law_set_random_seed(131351)
iuid = spde.compute(dat)
dat.display(dbfmt)

## Kriging

In [ ]:
spdeRes = gl.SPDE(model,grid,dat,gl.ESPDECalcMode.KRIGING,mesh,1)
iuid = spdeRes.compute(grid)
grid.display(dbfmt)

In [ ]:
gp.plot(grid)
gp.decoration(title="Estimation")

## Manually

### Projection Matrix: mesh to grid

In [ ]:
Pglg = gl.ProjMatrix(grid,mesh)
Aprojg = Pglg.toTL()

### Simulation

In [ ]:
Q = spdeRes.getPrecisionOpMatrix().getQ().toTL()
cholQ = cholesky(Q)

In [ ]:
u = np.random.normal(size = Q.shape[0])
gridExt["simuManual"] = cholQ.apply_Pt(cholQ.solve_Lt(1./np.sqrt(cholQ.D())*u))
gridExt.addSelection((gridExt["x1"]>0) & (gridExt["x2"]>0) & (gridExt["x1"]<1.) & (gridExt["x2"]<1.))

res = gp.plot(gridExt, "simuManual",useSel=False)
print(f"Variance = {round(np.var(gridExt['simuManual'][np.where(gridExt['NewSel']==1)]),4)}")

### Kriging

In [ ]:
Pgl = gl.ProjMatrix(dat,mesh)
Aproj = Pgl.toTL()

Qop = Q + 1/nugget * Aproj.T @ Aproj
cholQop =  cholesky(Qop)

kriging = cholQop.solve_A(Aproj.T @ (dat["spde*"]/nugget))

In [ ]:
grid["manually"] = Aprojg @ kriging

In [ ]:
ax = plt.scatter(grid["manually"],grid["*estim"],s=1)
p = plt.plot([-3,3],[-3,3],c="r")

## Likelihood

Manually with Cholesky vs. matrix-free approach with SPDE api.

In [ ]:
def solveMat(cholQop,x):
    return cholQop.solve_A(x)

def invSigma(sigma2,Aproj,cholQop,x):
    return 1./sigma2 * (x - 1./sigma2 * Aproj @ solveMat(cholQop, Aproj.T @ x))

def detQ(cholQ):
    return cholQ.logdet()

x = dat["spde"]
ones = np.ones_like(x)
invSigmaOnes = invSigma(nugget,Aproj,cholQop,ones)
mu  = np.sum(x * invSigmaOnes)/np.sum( ones * invSigmaOnes) 
quad = np.sum((x-mu)*invSigma(nugget,Aproj,cholQop,x-mu))
logdet = len(x) * np.log(nugget) - detQ(cholQ) + detQ(cholQop)

print(f"logdet_chol = {round(logdet,4)}")
print(f"quad_chol = {round(quad,4)}")
print(f"like_chol = {round(-0.5 * (quad + logdet + len(x) * np.log(2. * np.pi)),4)}")

In [ ]:
a_quad = spdeRes.computeQuad()
print(f"-> Relative difference quadratic = {round(100*(a_quad-quad)/quad,2)}%")

In [ ]:
pcm = spdeRes.getPrecisionKrig()
a_op = detQ(cholQop)
b_op = pcm.computeLogDetOp(1)
print(f"log_det_op_chol = {round(a_op,4)}")
print(f"log_det_op_api = {round(b_op,4)}")
print(f"-> Relative difference = {round(100*(b_op-a_op)/a_op, 2)}%")

In [ ]:
a_one = detQ(cholQ)
b_one = pcm.computeLogDetQ(10)
print(f"log_det_Q_chol = {round(a_one,4)}")
print(f"log_det_Q_api = {round(b_one,4)}")
print(f"-> Relative difference = {round(100*(b_one-a_one)/a_one,2)}%")

In [ ]:
a = -0.5 * (quad + logdet + len(x) * np.log(2. * np.pi))
print(f"likelihood api = {round(a,4)}")

model.setDriftIRF()
spdeLL = gl.SPDE(model,grid,dat,gl.ESPDECalcMode.KRIGING,mesh,1)
b = spdeLL.computeLogLikelihood(100)
print(f"likelihood_chol = {round(b,4)}")
print(f"-> Relative Difference = {round(100*(b-a)/a,2)}%")

In [ ]:
b2 = gl.logLikelihoodSPDE(dat,model,None,mesh,1)
print(f"likelihood by API {round(b2,4)}")